# HW7 (20')

<font size='4'>
    
Please submit your assignment as an HTML or PDF file.

Print your name (First and Last) below.

In [1]:
print('Brynna Garden')

Brynna Garden


## General Remarks

<font size='4'>

- In this HW, you will start to **write your own functions** to achieve a specific task.
- The prompt to HW is **no longer specific** to each step. 
- I only describe the purpose of the task and provide an empty placeholder for each function. <br>
- It is your own responsibility to decompose the task and fill in the body of the function. <br>
- You will have one dataset to test.
- You only work on code sections starting with `Write youw own code`.
- Feel free to ask me questions or stop by OH. I am sure you will!

## Background on the study and dataset

<font size='4'>
    
- I studied statistical methodology on EEG with applications to Brain-Computer Interfaces as my dissertation topic.
- The particular modality is P300 Event-Related Response (ERP) based Brain-Computer Interface (BCI).
- This is used for ordinary communications when users cannot type words due to a physical disability, such as ALS or stroke.
- Users are asked to wear an EEG cap, sit in front of a virtual screen, and respond to the stimuli on the virtual screen.
    - <img src="figures/p300-eeg-bci-illustration.jpg" alt="drawing" width="800"/>
    - YouTube Link: https://www.youtube.com/watch?v=XIr2cRKFolY&t=1s (Data collection starts from 2:16).
- The biological mechanism is that when the human brain is exposed to a relevant but rare external stimuli, the brain elicits a so-called (target) P300 ERP that can be captured via the EEG recording, otherwise, when the human brain is exposued to a irrelevant (but regular) external stimuli, the brain does not elicit this P300 ERP. To simplify, we call it an non-target P300 ERP.
- The conventional framework is shown
    - <img src="figures/bci_conventional_framework.png" alt="drawing" width="800"/>
    - The statistical problem is simply a **binary classification** problem.
- The resulting dataset contains a truncated EEG signal matrix of which dimension is (3420, 400) and labeled vector of which dimension is (3420, 1) telling whether it is a target or non-target stimuli.
    - <img src="figures/eeg_signal_data_structure.png" alt="drawing" width="800"/>
    - Here, 3420 refers to the total number of stimuli, while 400 is the total feature length.
    - The truncated dataset has 16 electrode, and each electrode has a 25 time points corresponding to the first 800 ms after each stimulus.
    - <img src="figures/electrode_location.png" alt="drawing" width="400"/>
- In HW 7, you are going to **validate the signal component between target and non-target ERPs**, which is an important descriptive step right after signal processing and feature extraction. In later HWs, you will have the chance to explicitly perform signal processing and feature extraction, perform binary classification tasks using various ML methods, and convert them into character-level probability calculation.

In [2]:
# import relevant packages
import os
import numpy as np
import scipy.io as sio # This will be used to load an MATLAB file
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as bpdf # This will be used to create a PDF to store multiple plots in the same file
# You may not use this function for HW7, but when we start to move everything to .py file, you will use it.

## Global constants and parent directory (1')

<font size='4'>

- You do not need to modify the global parameters.
- You need to change your parent directory to your own, preferably `.../GitHub/BIOS-584`.
- If you **fetch origins**, you will find a dataset called `K114_001_BCI_TRN_Truncated_Data_0.5_6.mat` under the `data` folder.

In [3]:
bp_low = 0.5
bp_upp = 6
electrode_num = 16
electrode_name_ls = ['F3', 'Fz', 'F4', 'T7', 'C3', 'Cz', 'C4', 'T8', 'CP3', 'CP4', 'P3', 'Pz', 'P4', 'PO7', 'PO8', 'Oz']

parent_dir = '/Users/tma33/Library/CloudStorage/OneDrive-EmoryUniversity/Emory/Rollins SPH/2025/BIOS-584/python_proj'
parent_data_dir = '{}/data'.format(parent_dir)
time_index = np.linspace(0, 800, 25) # This is a hypothetic time range up to 800 ms after each stimulus.

subject_name = 'K114'
session_name = '001_BCI_TRN'

<font size='4'>

- Use `os.mkdir()` and `os.path.exists()` to create a new folder called `subject_name` under your `parent_dir`.
- Make sure to use `if` to check whether the proposed directory exists because Python will yield an error if so.
- For HW7, you do not have to export anything outside Jupyter notebook. That will be your HW8.
- But You should still create a new directory below. (1')

In [4]:
# Write your own code:
#Seeing data path
parent_dir = os.getcwd()
print(parent_dir)
#Creating new folder called subject_name under partner_dir
subject_name = "{}/data/hw7".format(parent_dir)
#Making sure to use if to check whether the proposed directory exists
if not os.path.exists(subject_name):
    os.mkdir(subject_name)
print(subject_name) #It worked!

/Users/brynnagarden/Documents/GitHub/BIOS-584
/Users/brynnagarden/Documents/GitHub/BIOS-584/data/hw7


## Load the MATLAB .mat file (4')

<font size='4'>

- Use `sio.loadmat()` function to load the dataset and rename it as `eeg_trunc_obj`. (1')

In [5]:
# Write your own code:
#Creating data path to follow to read in the dataset
dataset_path = '/Users/brynnagarden/Documents/GitHub/BIOS-584/data/K114_001_BCI_TRN_Truncated_Data_0.5_6.mat'
#Loading in dataset and renaming it to eeg_trunc_obj
eeg_trunc_obj = sio.loadmat(dataset_path)
print(eeg_trunc_obj) #Making sure it worked

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Wed Jun 22 18:05:34 2022', '__version__': '1.0', '__globals__': [], 'Code': array([[ 1],
       [10],
       [11],
       ...,
       [ 3],
       [ 6],
       [ 5]], dtype=uint8), 'IndexBegin': array([[  154],
       [  159],
       [  164],
       ...,
       [19255],
       [19260],
       [19265]], dtype=uint16), 'IndexTag': array([[  154],
       [  159],
       [  164],
       ...,
       [19255],
       [19260],
       [19265]], dtype=uint16), 'LetterTable': array([[array(['A'], dtype='<U1')],
       [array(['B'], dtype='<U1')],
       [array(['C'], dtype='<U1')],
       [array(['D'], dtype='<U1')],
       [array(['E'], dtype='<U1')],
       [array(['F'], dtype='<U1')],
       [array(['G'], dtype='<U1')],
       [array(['H'], dtype='<U1')],
       [array(['I'], dtype='<U1')],
       [array(['J'], dtype='<U1')],
       [array(['K'], dtype='<U1')],
       [array(['L'], dtype='<U1')],
       [array(['M'], dtype='<

<font size='4'>

- Although we haven't covered `dict` data type yet, you have the chance to use it in practice.
- Print out `eeg_trunc_obj.keys()` and find the name `Signal` and `Type`. Those are the only two things we will use for HW7. (1')
- We extract information from `dict` using the syntax `variable_name = dict_obj['key_name']`.
- Rename the variable associated with `Signal` to `eeg_trunc_signal` and `Type` as `eeg_trunc_type`. (1')
- Print out their shapes respectively. For `eeg_trunc_type`, use `np.squeeze()` to convert it to a 1d-array. Make sure to spell out the correct `axis` parameter.
- What does `eeg_trunc_type` contain? (1')

In [6]:
# Write your own code:
#Printing out eeg_trunc_obj.keys() to find Signal and Type
print(eeg_trunc_obj.keys())
#Renaming variables with Signal to eeg_trunc_signal and Type as eeg_trunc_type
eeg_trunc_signal = eeg_trunc_obj['Signal']
eeg_trunc_type = eeg_trunc_obj['Type']
#Printing out their shapes respectively
print(eeg_trunc_signal.shape)
print(eeg_trunc_type.shape)
#Converting eeg_trunc_type to a 1d-array
eeg_trunc_type = np.squeeze(eeg_trunc_type, axis=-1)
#Printing out shape after the squeeze
print(eeg_trunc_type.shape)
#Looking at the first 10 of eeg_trunc_type
print(eeg_trunc_type[:10])
#Q: What does eeg_trunc_type contain? 
#A: eeg_trunc_type contains a 1D array of integers that categorize each time point of the EEG signal as a target (1) or non-target (-1) stimuli

dict_keys(['__header__', '__version__', '__globals__', 'Code', 'IndexBegin', 'IndexTag', 'LetterTable', 'Signal', 'Text', 'Type'])
(3420, 400)
(3420, 1)
(3420,)
[-1 -1 -1 -1 -1 -1 -1  1  1 -1]


## Write a function to compute type-specific sample mean and sample covariance (4')

<font size='4'>

- If you look at the individual signal segment, you cannot find anything interesting because the useful P300 ERP is overwhelmed in the noise.
- However, if you select all signals associated with target ERPs (i.e., `eeg_trunc_type == 1`) and take the average, you will find a clear pattern. Similarly do it for non-target ERPs.
- Please write a function called `produce_trunc_mean_cov` that takes the three parameters and output a list of five elements as follows:
- Make sure the dimensions of output arrays match with my description.

In [33]:
# Write your own code:
# Note to myself: keep r""" """ from the function. 
    r"""
    args:
    -----
        input_signal: 2d-array, (sample_size_len, feature_len)
        input_type: 1d-array, (sample_size_len,)
        E_val: integer, (number of electrodes)
    return:
    -----
        A list of 5 arrays including
            signal_tar_mean, (E_val, length_per_electrode)
            signal_ntar_mean, (E_val, length_per_electrode)
            signal_tar_cov, (E_val, length_per_electrode, length_per_electrode)
            signal_ntar_cov, (E_val, length_per_electrode, length_per_electrode)
            signal_all_cov, (E_val, length_per_electrode, length_per_electrode)
    note:
    -----
        descriptive mean and sample covariance statistics from real data
        In this case, E_val=16, length_per_electrode=25. 
        But you should pass them as arguments or calculate them inside the function.
    """
def produce_trun_mean_cov(input_signal, input_type, E_val):
    sample_size_len, feature_len = input_signal.shape
    sample_size_len = input_type.shape
    number_of_electrodes = E_val.int

    #Initailizing lists to store e_val & length_per_electrode
    signal_tar_mean_ls = []
    signal_ntar_mean_ls = []
    signal_tar_cov_ls = []
    signal_ntar_cov_ls = []
    signal_all_cov_ls = []

    #Converting lists to arrays 
    signal_tar_mean = np.array(signal_tar_mean_ls)
    signal_ntar_mean = np.array(signal_ntar_mean_ls)
    signal_tar_cov = np.array(signal_tar_cov_ls)
    signal_ntar_cov = np.array(signal_ntar_cov_ls)
    signal_all_cov = np.array(signal_all_cov_ls)

    return [signal_tar_mean, signal_ntar_mean, signal_tar_cov, signal_ntar_cov, signal_all_cov]

## Write a function to plot sample means (3') and sample covariances (3')

<font size='4'>

- After you extract the sample mean and sample covariance, you can visualize those to gain a better understanding.
- You will write two functions to visualize the sample mean (`plot_trunc_mean`) and sample covariance (`plot_trunc_cov`).

For sample mean:
- You are going to have 16 plots and put them into a 4x4 grid such that channels F3, C3, CP3, and P4 show up on the first, second, third, and fourth row, but all within the first column.
- In each subplot, you are going to plot target sample mean and non-target sample mean simulatenously and label them `Target` and `Non-Target` respectively.
- Use red for target sample mean and blue for non-target sample mean.
- Add `Time (ms)` to the label of x-axis, `Amplitude (muV)` to the label of y-axis for each subplot.
- Add the channel name to the title for each sub plot (Use `electrode_name_ls`)
- Add a subject name to the super title (shown on the top of the entire 4x4 grid).
- Display the final plot.

In [22]:
def plot_trunc_mean(
        eeg_tar_mean, eeg_ntar_mean, subject_name, time_index, E_val, electrode_name_ls,
        y_limit=np.array([-5, 8]), fig_size=(12, 12)
):
    r"""
    :param eeg_tar_mean:
    :param eeg_ntar_mean:
    :param subject_name:
    :param time_index:
    :param E_val:
    :param electrode_name_ls:
    :param y_limit: optional parameter, a list or an array of two numbers
    :param fig_size: optional parameter, a tuple of two numbers
    :return:
    """
    pass

<font size='4'>
For sample covariance:

- You also put 16 electrode-specific covariance plots and squeeze them into a 4x4 grid such that channels F3, C3, CP3, and P4 show up on the first, second, third, and fourth row, but all within the first column.
- You save **three** plots, **one for target signals, one for non-target, and one for all**.
- Use `np.meshgrid` and `plt.contourf()` to generate the plot. You are welcome to use other function. You can use whatever color palette but make sure to mark the color bar.
- For x-axis, the time index should increase from left to right, while for y-axis, the time index should increase from top to bottom.
- Add `Time (ms)` to the label of x- and y-axes.
- Add channel name to the title of each subplot.
- Add `Target`, `Non-target`, or `All` to the super title of each plot.
- Display the final three plots.

In [23]:
def plot_trunc_cov(
        eeg_cov, cov_type, time_index, subject_name, E_val, electrode_name_ls, fig_size=(14,12)
):
    pass

## Call your function and apply it to a particular participant K114. (5')

In [24]:
# Write your own code: (only call produce_trun_mean_cov in this code section)
'''
your output = produce_trun_mean_cov(your input)
'''

'\nyour output = produce_trun_mean_cov(your input)\n'

In [25]:
# Write your own code: (only call plot_trunc_mean in this code section)


In [26]:
# Write your own code: call plot_trunc_cov for target covariance only


In [27]:
# Write your own code: call plot_trunc_cov for non-target covariance only


In [28]:
# Write your own code: call plot_trunc_cov for all covariance only
